In [9]:
#%pip install python-dotenv
import numpy as np
import pandas as pd
import dotenv
import requests
import json
import os

In [10]:
url = 'https://httpbin.org/user-agent'
r = requests.get(url)
my_json = json.loads(r.text)
useragent = my_json['user-agent']
headers = {'User-Agent': useragent,
           'From': 'ljn5yms@virginia.edu'}

In [11]:
dotenv.load_dotenv()
newskey = os.getenv('newskey')

In [12]:
params = {'apiKey': newskey,
          'q': 'Bob Good'}
url = 'https://newsapi.org/v2/everything'
r = requests.get(url, headers=headers, params=params)
my_json = json.loads(r.text)
newsdf = pd.json_normalize(my_json, record_path = 'articles')
newsdf

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Brent D. Griffiths,Trump continues his revenge tour against pro-D...,Donald Trump hasn't forgotten that Rep. Bob Go...,https://www.businessinsider.com/trump-revenge-...,https://i.insider.com/6655e798a961b37edf3a3db1...,2024-05-28T14:47:14Z,Rep. Bob Good of Virginia had tried to appear ...,business-insider,Business Insider
1,Julia Manchester,MAGA civil war reaches climax in Bob Good primary,The MAGA civil war surrounding House Freedom C...,https://thehill.com/homenews/campaign/4726402-...,https://media.zenfs.com/en/the_hill_articles_3...,2024-06-17T21:22:08Z,The MAGA civil war surrounding House Freedom C...,the-hill,The Hill
2,None,"Bob Good vows to have recount, full investigat...",None,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-06-20T23:37:16Z,"If you click 'Accept all', we and our partners...",None,Yahoo Entertainment
3,"Oren Oppenheim, Jacob Steinberg","Tight Virginia GOP primary between Good, McGui...",A contentious and closely watched Republican U...,https://abcnews.go.com/Politics/tight-virginia...,https://i.abcnewsfe.com/a/00c4e1b4-b2eb-4d35-9...,2024-06-19T16:06:43Z,A contentious and closely watched Republican U...,abc-news,ABC News
4,None,Glastonbury festival 'was due to close' in the...,Emily Eavis says the festival was due to shut ...,https://www.bbc.com/news/articles/cz99nvngzdwo,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-06-10T01:15:40Z,Emily Eavis was speaking to Annie MacManus and...,None,BBC News
...,...,...,...,...,...,...,...,...,...
95,Molly McCloskey,The Detroit Pistons Were My Father’s Second Fa...,"Molly McCloskey remembers her father, Jack McC...",https://www.newyorker.com/magazine/2024/06/03/...,https://media.newyorker.com/photos/6650d628dc0...,2024-05-27T10:00:00Z,"One night, when I was a sophomore in college, ...",None,The New Yorker
96,Bradley Devlin,The Primary Race That Will Define Next Congress,Virginia’s Fifth Congressional District GOP pr...,https://www.theamericanconservative.com/the-pr...,https://www.theamericanconservative.com/wp-con...,2024-06-05T04:05:00Z,"For the coming weeks, the Fifth Congressional ...",the-american-conservative,The American Conservative
97,Bradley Devlin,Freedom Caucus Chair Under Fire After Trump En...,Former President Donald Trump has endorsed Vir...,https://www.theamericanconservative.com/freedo...,https://www.theamericanconservative.com/wp-con...,2024-05-29T04:05:00Z,The former President Donald Trump has made a k...,the-american-conservative,The American Conservative
98,Bradley Devlin,Local GOP Leaders Ask Trump to Reconsider Back...,State of the Union: In the aftermath of former...,https://www.theamericanconservative.com/local-...,https://www.theamericanconservative.com/wp-con...,2024-06-10T15:00:00Z,On the heels of former President Donald Trumps...,the-american-conservative,The American Conservative


In [13]:
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')


In [14]:
root = 'https://api.congress.gov/v3'
endpoint = '/member'
params = {'api_key': congresskey,
          'format': 'json',
          'currentMember': 'true',
          'limit': 250}
r = requests.get(root + endpoint,params=params,headers=headers)
r

<Response [200]>

In [15]:
members_df = pd.json_normalize(json.loads(r.text), record_path = ['members'])
bobgoodinfo = members_df.query('state=="Virginia" & district==5')
bobgood_id = bobgoodinfo['bioguideId'][15]
bobgood_id

'G000595'

In [16]:
endpoint = f'/member/{bobgood_id}/sponsored-legislation'
r = requests.get(root + endpoint,params=params,headers=headers)
r

<Response [200]>

In [17]:
bobgood_bills = pd.json_normalize(json.loads(r.text),record_path = 'sponsoredLegislation')
bobgood_bills

,congress,introducedDate,number,title,type,url,latestAction.actionDate,latestAction.text,policyArea.name,latestAction.actionTime,amendmentNumber
0,118,2024-05-23,8534,Protecting Student Athletes’ Economic Freedom Act,HR,https://api.congress.gov/v3/bill/118/hr/8534?f...,2024-06-13,Ordered to be Reported (Amended) by the Yeas a...,None,NaN,NaN
1,118,2024-05-16,8432,Protect Our Police Act,HR,https://api.congress.gov/v3/bill/118/hr/8432?f...,2024-05-16,Referred to the House Committee on the Judiciary.,None,NaN,NaN
2,118,2024-05-08,8306,Silencers Help Us Save Hearing Act,HR,https://api.congress.gov/v3/bill/118/hr/8306?f...,2024-05-08,"Referred to the Committee on Ways and Means, a...",None,NaN,NaN
3,118,2024-04-30,8178,Apprenticeship Freedom Act,HR,https://api.congress.gov/v3/bill/118/hr/8178?f...,2024-04-30,Referred to the House Committee on Education a...,None,NaN,NaN
4,118,2024-04-19,8091,To prohibit Federal funding of National Public...,HR,https://api.congress.gov/v3/bill/118/hr/8091?f...,2024-04-19,Referred to the House Committee on Energy and ...,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
101,117,2021-02-11,992,No Taxpayer Funding for the Chinese Communist ...,HR,https://api.congress.gov/v3/bill/117/hr/992?fo...,2021-02-11,"Referred to the Committee on Science, Space, a...",International Affairs,NaN,NaN
102,117,2021-02-11,991,Preserving Safety Net Integrity Act of 2021,HR,https://api.congress.gov/v3/bill/117/hr/991?fo...,2021-04-23,Referred to the Subcommittee on Immigration an...,Immigration,NaN,NaN
103,117,2021-02-03,754,Protecting Religion from Government Act of 2021,HR,https://api.congress.gov/v3/bill/117/hr/754?fo...,2021-03-22,Referred to the Subcommittee on the Constituti...,"Civil Rights and Liberties, Minority Issues",NaN,NaN
104,117,2021-02-03,755,Right to Earn a Living Act of 2021,HR,https://api.congress.gov/v3/bill/117/hr/755?fo...,2021-02-03,Referred to the House Committee on Oversight a...,Government Operations and Politics,NaN,NaN


In [18]:
bobgood_bills['title'].tolist()

['Protecting Student Athletes’ Economic Freedom Act',
 'Protect Our Police Act',
 'Silencers Help Us Save Hearing Act',
 'Apprenticeship Freedom Act',
 'To prohibit Federal funding of National Public Radio and the use of Federal funds to acquire radio content.',
 'Iran China Accountability Act of 2024',
 'Every Town A Border Town Act',
 'Denouncing calls for a cease-fire in Gaza.',
 'Blue Ridge Fire Safety Act',
 'To prohibit the Administrator of the Environmental Protection Agency from finalizing, implementing, or enforcing the proposed rule related to revisions to the air emissions reporting requirements, and for other purposes.',
 'No Regulation Through Litigation Act of 2023',
 'Veterans Education is Timeless Act of 2023',
 'No American Climate Corps Act',
 'Expressing support for the designation of October 12, 2023, as "National Loggers Day".',
 'No Discrimination in My Benefits Act',
 'No Vaccine Mandates in Higher Education Act',
 'Empowering Local Curriculum Act',
 'Empowering 

In [19]:
dotenv.load_dotenv()
opensecretskey = os.getenv('opensecretskey')
root = 'https://www.opensecrets.org/api'
endpoint = '/?method=getLegislators'
params = {'apikey': opensecretskey,
          'id': 'VA',
          'output': 'json'}
r = requests.get(root + endpoint,params=params,headers=headers)
r

<Response [200]>

In [20]:
json.loads(r.text)

{'response': {'legislator': [{'@attributes': {'cid': 'N00029459',
     'firstlast': 'Rob Wittman',
     'lastname': 'Wittman',
     'party': 'R',
     'office': 'VA01',
     'gender': 'M',
     'first_elected': '2007',
     'exit_code': '0',
     'comments': '',
     'phone': '202-225-4261',
     'fax': '202-225-4382',
     'website': 'http://wittman.house.gov',
     'webform': 'https://wittman.house.gov/contact-form/',
     'congress_office': '2055 Rayburn House Office Building',
     'bioguide_id': 'W000804',
     'votesmart_id': '58133',
     'feccandid': 'H8VA01147',
     'twitter_id': 'RobWittman',
     'youtube_url': 'https://youtube.com/RobWittman',
     'facebook_id': 'RepRobWittman',
     'birthdate': '1959-02-03'}},
   {'@attributes': {'cid': 'N00048315',
     'firstlast': 'Jen Kiggans',
     'lastname': 'Kiggans',
     'party': 'R',
     'office': 'VA02',
     'gender': 'F',
     'first_elected': '2022',
     'exit_code': '0',
     'comments': '',
     'phone': '',
     'fax

In [34]:
memberdf = pd.json_normalize(json.loads(r.text), record_path = ['response','legislator'])
memberdf.columns = [x.replace('@attributes.','') for x in memberdf.columns]
good_cid = memberdf.query('lastname=="Good"')['cid'].tolist()[0]
good_cid

'N00045557'

In [35]:
endpoint = '/?method=candContrib'
params = {'apikey': opensecretskey, 
          'cid': good_cid, 
          'cycle': 2022, 
          'output': 'json'
          }
r = requests.get(root + endpoint,params=params,headers=headers)
r

<Response [200]>

In [37]:
json.loads(r.text)

{'response': {'contributors': {'@attributes': {'cand_name': 'Bob Good (R)',
    'cid': 'N00045557',
    'cycle': '2022',
    'origin': 'OpenSecrets',
    'source': 'https://www.opensecrets.org/members-of-congress/contributors?cid=N00045557&cycle=2022',
    'notice': "The organizations themselves did not donate, rather the money came from the organization's PAC, its individual members or employees or owners, and those individuals' immediate families."},
   'contributor': [{'@attributes': {'org_name': 'House Freedom Fund',
      'total': '33200',
      'pacs': '10000',
      'indivs': '23200'}},
    {'@attributes': {'org_name': 'Rst Marketing',
      'total': '13500',
      'pacs': '0',
      'indivs': '13500'}},
    {'@attributes': {'org_name': 'Dimensional Fund Advisors',
      'total': '11600',
      'pacs': '0',
      'indivs': '11600'}},
    {'@attributes': {'org_name': 'Kinsale Capital Group',
      'total': '11600',
      'pacs': '0',
      'indivs': '11600'}},
    {'@attributes':